In [ ]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="udacity-project")
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute

compute_config = AmlCompute.provisioning_configuration(vm_size="Standard_D2_V2", max_nodes=4)
aml_compute = ComputeTarget.create(ws, "the-cluster", compute_config)
aml_compute.wait_for_completion(show_output=True)

In [ ]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
import os

# Specify parameter sampler
ps = RandomParameterSampling({"C": uniform(0, 10), "max_iter": choice(20, 40, 60, 80, 100)})

# Specify a Policy
policy = BanditPolicy(evaluation_interval=1, slack_factor=None, slack_amount=None, delay_evaluation=0)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn("train.py", compute_target = cpu_cluster)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(hyperparameter_sampling = ps,
                                     policy = policy,
                                     estimator = est,
                                     primary_metric_name = "Accuracy",
                                     primary_metric_goal = PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs = 25,
                                     max_concurrent_runs = 4)

In [ ]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hyperdrive_run = exp.submit(hyperdrive_config)
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output = True)

In [ ]:
import joblib
# Get your best run and save the model from that run.
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_details()["runDefinition"]["arguments"]

print("Best Run:" + best_run.id)
print("Accuracy:" + best_run_metrics["Accuracy"])
print("C:" + parameter_values[1])
print("Maximum Interation:" + parameter_values[3])

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

ds = TabularDatasetFactory.from_delimited_files("https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv")

In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

In [ ]:
from azureml.train.automl import AutoMLConfig
automl_config = AutoMLConfig(experiment_timeout_minutes=30,
                             task="classification",
                             primary_metric="accuracy",
                             training_data=ds,
                             label_column_name="y",
                             n_cross_validations=2)

In [2]:
# Submit your automl run

automl_exp = Experiment(ws, "the-experiment")
automl_run = automl_exp.submit(config = automl_config, show_output = True)

In [ ]:
# Retrieve and save your best automl model.

automl_best_run, automl_best_model = run.get_output()
best_run.register_model(model_name = "best_model_aml.pkl", model_path = './outputs/')